# SVM, KNN and RF evaluation on the dataset in the filtered time domain

## Setting up the evaluation

### Importing the necessary libraries and setting things up

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import sys
import tabulate
import matplotlib.pyplot as plt

In [2]:
REPO_ROOT_DIR = "../../../"
sys.path.append(os.path.dirname(REPO_ROOT_DIR))
import src

In [3]:
from src.features import Filter

In [4]:
from src.models import SVM
from src.models import KNN
from src.models import RF

In [5]:
csv_path = os.path.join(REPO_ROOT_DIR, "data/raw/csv/")

## Setting up the data

### Reading the dataset

In [6]:
dataset = src.data.mit_bih_dataset.MIT_BIH_Dataset(path=csv_path)

In [7]:
X_data = dataset.get_X_data()
y_data = dataset.get_y_data()

### Filtering the X data

In [8]:
filter = Filter.Filter(X_data)
filter.denoise_data()
X_data = filter.get_clean_data()

/usr/local/lib/python3.8/dist-packages/pywt/_multilevel.py:43: UserWarning: Level value of 10 is too high: all coefficients will experience boundary effects.
  warnings.warn(


### Creating the train and test set

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state=42)

## Evaluating the models

## SVM

In [10]:
svm_model = SVM.SVM_Model(X_train, y_train)
svm_model.fit()
svm_model.predict(X_test)
svm_metrics = svm_model.eval_model(y_test)

In [11]:
print("F-medida:", round(svm_metrics[0], 4), "| Acurácia:", round(svm_metrics[1], 4))

F-medida: 0.9311 | Acurácia: 0.9299


## RF

In [12]:
rf_model = RF.RF_Model(X_train, y_train)
rf_model.fit()
rf_model.predict(X_test)
rf_metrics = rf_model.eval_model(y_test)

In [13]:
print("F-medida:", round(rf_metrics[0], 4), "| Acurácia:", round(rf_metrics[1], 4))

F-medida: 0.9514 | Acurácia: 0.951


## KNN

In [14]:
knn_model = KNN.KNN_Model(X_train, y_train)
knn_model.fit()
knn_model.predict(X_test)
knn_metrics = knn_model.eval_model(y_test)

In [15]:
print("F-medida:", round(knn_metrics[0], 4), "| Acurácia:", round(knn_metrics[1], 4))

F-medida: 0.9337 | Acurácia: 0.9334
